In [ ]:
import sqlite3
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
con_rev = sqlite3.connect('review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)
df_rev.head()

In [ ]:
df_rev.created_at = df_rev.apply(lambda row : datetime.strptime(row.created_at, '%Y-%m-%dT%H:%M:%S.000Z'), axis=1,)

In [ ]:
df_rev.head()

In [ ]:
df_rev

In [ ]:
greater_one = df_rev.groupby(by='user_id').filter(lambda x: x['rating'].count() > 5)

In [ ]:
sb.boxplot(greater_one.user_id.value_counts(), orient='h')

In [ ]:
greater_one.user_id.value_counts()